In [1]:
import sys
import sqlite3
import pandas as pd
import numpy as np
import re

In [2]:
## 201812~현재까지 유효
yyyy = '2019'
mm = '12'
reports = pd.read_excel('../data/financial_reports/%s년%s월확정실적/%s%s(한글).xlsx'%(yyyy, mm, yyyy, mm),
                        sheet_name='주재무제표', header=[1, 2], dtype={'산업코드':np.str})
columns = [t[0] if str(t[1]).startswith('Unnamed') else "%s-%s"%(re.sub('\n', '', t[0]), t[1]) for t in reports.keys()]
reports.columns = columns

# 필요한 필드만 추출
cols = ('시장', '종목코드', '회사명', '회계기준', '산업코드', '산업명', '자산총계', '유동자산', '자본총계', '부채총계',
        '매출액-', '매출원가', '매출총이익', '영업이익(보고서기재)', '당기순이익', '영업활동으로인한현금흐름', '부채비율')
reports = reports[[k for k in reports.keys() if k.startswith(cols) and '비교' not in k and '3개월' not in k]]
reports = reports[[k for k in reports.keys() if '-' not in k or '%s%s'%(yyyy, mm) in k]]
reports['종목코드'] = reports['종목코드'].str[1:]
reports = reports[~reports['회사명'].str.contains('스팩')]
reports = reports[~reports['회사명'].str.endswith('우')]
reports = reports[~reports['회사명'].str.endswith('우B')]

# 필드명 정제
cols = reports.keys()
cols = [re.sub('\(.*\)', '', k) for k in cols] # (보고서기재) 제거
cols = [re.sub('활동으로인한', '', k) for k in cols] # 영업현금흐름 으로 정제
cols = [re.sub('\/누적', '', k) for k in cols] # 201906/누적 => 201906 으로 정제
reports.columns = cols

In [3]:
reports.head()

,시장,종목코드,회사명,회계기준,산업코드,산업명,자산총계-201912,유동자산-201912,부채총계-201912,자본총계-201912,매출액-201912,매출원가-201912,매출총이익-201912,영업이익-201912,당기순이익-201912,영업현금흐름-201912,부채비율-201912
0,KS,000020,동화약품,연결,32100,의료용 물질 및 의약품 제조업,376028553,2.354570e+08,75153959,300874594,3.071500e+08,1.854183e+08,121731711.0,9917746,9393855,1.328497e+07,24.978500
1,KS,000040,KR모터스,연결,33100,기타 운송장비 제조업,144081160,5.269909e+07,108871143,35210017,1.325793e+08,1.335669e+08,-987674.0,-24650440,-41041685,-7.078682e+06,309.205028
2,KS,000050,경방,연결,74700,소매업; 자동차 제외,1370742491,2.648610e+08,629189292,741553199,3.438921e+08,2.443960e+08,99496064.0,32020530,43621631,6.687528e+07,84.847492
3,KS,000060,메리츠화재,연결,116601,보험업,23215663095,NaN,20775120387,2440542707,NaN,NaN,NaN,352756185,301296160,1.026132e+09,NaN
4,KS,000070,삼양홀딩스,연결,137100,전문 서비스업,3589513891,1.259478e+09,1560007367,2029506524,2.488563e+09,2.025132e+09,463430768.0,79376318,62043854,2.189381e+08,76.866339


In [4]:
reports.keys()

Index(['시장', '종목코드', '회사명', '회계기준', '산업코드', '산업명', '자산총계-201912',
       '유동자산-201912', '부채총계-201912', '자본총계-201912', '매출액-201912',
       '매출원가-201912', '매출총이익-201912', '영업이익-201912', '당기순이익-201912',
       '영업현금흐름-201912', '부채비율-201912'],
      dtype='object')

In [5]:
# company 기본정보 추출
companies = reports[['종목코드', '시장', '회사명', '산업코드', '산업명']]
companies.columns = ['code', 'market', 'company', 'ind_code', 'industry']
companies.head()

,code,market,company,ind_code,industry
0,000020,KS,동화약품,32100,의료용 물질 및 의약품 제조업
1,000040,KS,KR모터스,33100,기타 운송장비 제조업
2,000050,KS,경방,74700,소매업; 자동차 제외
3,000060,KS,메리츠화재,116601,보험업
4,000070,KS,삼양홀딩스,137100,전문 서비스업


In [6]:
# company 정보 DB Insert
conn = sqlite3.connect("../database/quantative_investing.db")
for idx, company in companies.iterrows():
    try:
        conn.execute('''
            insert into companies (code, market, company, ind_code, industry)
                values ('%s', '%s', '%s', '%s', '%s')'''%(company.code, company.market, company.company, company.ind_code, company.industry))
    except sqlite3.IntegrityError:
        pass # 이미 인서트 된 종목코드 오류 무시
    except:
        print(sys.exc_info())
        pass
conn.commit()
conn.close()

In [7]:
conn = sqlite3.connect("../database/quantative_investing.db")
cur = conn.cursor()
cur.execute("select count(*) from companies")
rows = cur.fetchall()
cur.close()
conn.close()
print(rows)

[(2040,)]


In [8]:
# 제무제표 항목명과 DB 필드명 매핑
dic_db2name = {
    "code":"종목코드",
    "total_assets":"자산총계",
    "current_assets":"유동자산",
    "total_liabilities":"부채총계",
    "total_equity":"자본총계",
    "net_sales":"매출액",
    "cost_of_sales":"매출원가",
    "gross_profit":"매출총이익",
    "operating_income":"영업이익",
    "ongoing_operating_income":"당기순이익",
    "cash_flows_from_operatings":"영업현금흐름",
    "current_ratio":"유동비율",
    "debt_to_equity_ratio":"부채비율"
#     roa float, -- 당기순이익 / 자산총계
#     gpa float, -- 매출총이익 / 자산총계
}
print(dic_db2name)
dic_name2db = {dic_db2name[key]:key for key in dic_db2name}
print(dic_name2db)

{'code': '종목코드', 'total_assets': '자산총계', 'current_assets': '유동자산', 'total_liabilities': '부채총계', 'total_equity': '자본총계', 'net_sales': '매출액', 'cost_of_sales': '매출원가', 'gross_profit': '매출총이익', 'operating_income': '영업이익', 'ongoing_operating_income': '당기순이익', 'cash_flows_from_operatings': '영업현금흐름', 'current_ratio': '유동비율', 'debt_to_equity_ratio': '부채비율'}
{'종목코드': 'code', '자산총계': 'total_assets', '유동자산': 'current_assets', '부채총계': 'total_liabilities', '자본총계': 'total_equity', '매출액': 'net_sales', '매출원가': 'cost_of_sales', '매출총이익': 'gross_profit', '영업이익': 'operating_income', '당기순이익': 'ongoing_operating_income', '영업현금흐름': 'cash_flows_from_operatings', '유동비율': 'current_ratio', '부채비율': 'debt_to_equity_ratio'}


In [9]:
reports.keys()

Index(['시장', '종목코드', '회사명', '회계기준', '산업코드', '산업명', '자산총계-201912',
       '유동자산-201912', '부채총계-201912', '자본총계-201912', '매출액-201912',
       '매출원가-201912', '매출총이익-201912', '영업이익-201912', '당기순이익-201912',
       '영업현금흐름-201912', '부채비율-201912'],
      dtype='object')

In [10]:
# 지난분기 이름과 필드명 조정
yyyymm = list(set([re.sub('^.*-', '', key) for key in reports.keys() if '-' in key]))
print(yyyymm)
fields = list(set([re.sub('-.*$', '', key) for key in reports.keys() if '-' in key]))
print(fields)

joins = ['%s-%s'%(f, ym) for ym in yyyymm for f in fields]
print(joins)

['201912']
['매출액', '자본총계', '영업이익', '유동자산', '자산총계', '당기순이익', '부채비율', '매출원가', '매출총이익', '영업현금흐름', '부채총계']
['매출액-201912', '자본총계-201912', '영업이익-201912', '유동자산-201912', '자산총계-201912', '당기순이익-201912', '부채비율-201912', '매출원가-201912', '매출총이익-201912', '영업현금흐름-201912', '부채총계-201912']


In [11]:
# 지난 분기 데이터를 행으로 바꾸어 최종 머지
reports[[f for f in joins if f in reports.keys()]]
db_reports = []
for ym in yyyymm:
    jf = ['종목코드'] + ['%s-%s'%(f, ym) for f in fields]
    ym_report = reports[[f for f in jf if f in reports.keys()]]
    ym_report.columns = [re.sub('-.*$', '', key) for key in ym_report.keys()]
    ym_report.insert(1, 'rdate', ym+'31')
    db_reports.append(ym_report)
merged_reports = pd.concat(db_reports)
print(len(merged_reports))
merged_reports.head()

2040


,종목코드,rdate,매출액,자본총계,영업이익,유동자산,자산총계,당기순이익,부채비율,매출원가,매출총이익,영업현금흐름,부채총계
0,000020,20191231,3.071500e+08,300874594,9917746,2.354570e+08,376028553,9393855,24.978500,1.854183e+08,121731711.0,1.328497e+07,75153959
1,000040,20191231,1.325793e+08,35210017,-24650440,5.269909e+07,144081160,-41041685,309.205028,1.335669e+08,-987674.0,-7.078682e+06,108871143
2,000050,20191231,3.438921e+08,741553199,32020530,2.648610e+08,1370742491,43621631,84.847492,2.443960e+08,99496064.0,6.687528e+07,629189292
3,000060,20191231,NaN,2440542707,352756185,NaN,23215663095,301296160,NaN,NaN,NaN,1.026132e+09,20775120387
4,000070,20191231,2.488563e+09,2029506524,79376318,1.259478e+09,3589513891,62043854,76.866339,2.025132e+09,463430768.0,2.189381e+08,1560007367


In [14]:
for field in fields:
    if field.endswith('율'): continue
    merged_reports[field] = merged_reports[field] / 1000 # 천원단위를 백만원단위로 변경

In [15]:
merged_reports.head()

,종목코드,rdate,매출액,자본총계,영업이익,유동자산,자산총계,당기순이익,부채비율,매출원가,매출총이익,영업현금흐름,부채총계
0,000020,20191231,0.307150,300874.594,9917.746,235456.988,3.760286e+05,9393.855,24.978500,185418.315,121731.711,13284.966,7.515396e+04
1,000040,20191231,0.132579,35210.017,-24650.440,52699.087,1.440812e+05,-41041.685,309.205028,133566.936,-987.674,-7078.682,1.088711e+05
2,000050,20191231,0.343892,741553.199,32020.530,264861.016,1.370742e+06,43621.631,84.847492,244396.020,99496.064,66875.285,6.291893e+05
3,000060,20191231,NaN,2440542.707,352756.185,NaN,2.321566e+07,301296.160,NaN,NaN,NaN,1026131.749,2.077512e+07
4,000070,20191231,2.488563,2029506.524,79376.318,1259477.808,3.589514e+06,62043.854,76.866339,2025132.241,463430.768,218938.076,1.560007e+06


In [16]:
# DB 필드명과 동일하게 DF 필드명 변경
keys = merged_reports.keys()
print(list(keys))
keys2 = [dic_name2db[k] if k in dic_name2db else k for k in keys ]
print(keys2)
merged_reports.columns = keys2
merged_reports.head()

['종목코드', 'rdate', '매출액', '자본총계', '영업이익', '유동자산', '자산총계', '당기순이익', '부채비율', '매출원가', '매출총이익', '영업현금흐름', '부채총계']
['code', 'rdate', 'net_sales', 'total_equity', 'operating_income', 'current_assets', 'total_assets', 'ongoing_operating_income', 'debt_to_equity_ratio', 'cost_of_sales', 'gross_profit', 'cash_flows_from_operatings', 'total_liabilities']


,code,rdate,net_sales,total_equity,operating_income,current_assets,total_assets,ongoing_operating_income,debt_to_equity_ratio,cost_of_sales,gross_profit,cash_flows_from_operatings,total_liabilities
0,000020,20191231,0.307150,300874.594,9917.746,235456.988,3.760286e+05,9393.855,24.978500,185418.315,121731.711,13284.966,7.515396e+04
1,000040,20191231,0.132579,35210.017,-24650.440,52699.087,1.440812e+05,-41041.685,309.205028,133566.936,-987.674,-7078.682,1.088711e+05
2,000050,20191231,0.343892,741553.199,32020.530,264861.016,1.370742e+06,43621.631,84.847492,244396.020,99496.064,66875.285,6.291893e+05
3,000060,20191231,NaN,2440542.707,352756.185,NaN,2.321566e+07,301296.160,NaN,NaN,NaN,1026131.749,2.077512e+07
4,000070,20191231,2.488563,2029506.524,79376.318,1259477.808,3.589514e+06,62043.854,76.866339,2025132.241,463430.768,218938.076,1.560007e+06


In [17]:
# company 정보 DB Insert
conn = sqlite3.connect("../database/quantative_investing.db")
for idx, report in merged_reports.iterrows():
    columns = ', '.join(report.keys())
    placeholders = ', '.join('?' * len(report.values))
    sql = 'INSERT INTO reports ({}) VALUES ({})'.format(columns, placeholders)
    
    try:
        conn.execute(sql, report.values)
    except sqlite3.IntegrityError:
        pass # 이미 인서트되어 발생하는 오류 무시
    except:
        print(sys.exc_info())
        pass
conn.commit()
conn.close()

In [18]:
conn = sqlite3.connect("../database/quantative_investing.db")
cur = conn.cursor()
cur.execute("select count(*) from reports")
rows = cur.fetchall()
cur.close()
conn.close()
print(rows)

[(2040,)]
